In [1]:

# coding: utf-8

# In[26]:



# -*- coding:utf-8 -*-
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import SGD
from keras.layers import GRU
from keras.layers import MaxPooling1D
from keras.layers import Dropout


dropout_rate = 0.3


def mean_absolute_percentage_error(y_true, y_pred):
    y_true = y_true
    #print np.abs((y_true - y_pred) / y_true)
    m = np.abs((y_true - y_pred) / y_true)
    return m


def mape(n, label, prediction):
    j = 0
    m = 0

    #j.astype('float64')
    for i in range(0, n, 1):
        #mapen = mean_absolute_percentage_error(label[i], prediction[i])
        j=j+mean_absolute_percentage_error(label[i], prediction[i])[0]
    m=float(j/n)
    return m



train_dataset = read_csv('trainingA.csv',header=0,index_col=None)
train_values = train_dataset.values


test_dataset = read_csv('validationB.csv',header=0, index_col=None)
test_values = test_dataset.values

train_X, train_y = train_values[:, 1:4], train_values[:, 0]

test_X, test_y = test_values[:, 1:4], test_values[:, 0]


# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))

test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


print(train_X.shape[1], train_X.shape[2])

model = Sequential()
model.add(GRU(16, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True,activation='relu'))
model.add(Dropout(dropout_rate))
model.add(GRU(64,activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

optimizer = SGD(lr=0.001, momentum=0.9)
model.compile(loss='msle', optimizer=optimizer)
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=8, verbose=2, shuffle=False)


# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 0:]), axis=1)
inv_yhat = inv_yhat[:, 0]
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 0:]), axis=1)
inv_y = inv_y[:, 0]

l = len(test_y)
single_mape = mean_absolute_percentage_error(test_y, yhat)
for index in range(len(single_mape)):
    single_mape[index] = round(float(single_mape[index]),2)

para1 = []
para2 = []
para3 = []
for index in range(len(test_y)):
    para1.append(test_y[index][0])

for index in range(len(yhat)):
    para2.append(yhat[index][0])

for index in range(len(single_mape)):
    para3.append(single_mape[index][0])


Data = {'reality':para1,'predict': para2, 'mape': para3}
df = DataFrame(Data, columns=["reality", "predict", "mape"])
df.to_csv("experimentA_GRU_validation_result.csv", index=False)

average_mape = mape(l, test_y, yhat)
print('average mape is'+"%.4f" % average_mape)



C:\Users\shans\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(16032, 1, 3) (16032,) (1464, 1, 3) (1464,)
1 3
Epoch 1/50
 - 6s - loss: 1.5619
Epoch 2/50
 - 4s - loss: 0.1413
Epoch 3/50
 - 4s - loss: 0.1356
Epoch 4/50
 - 5s - loss: 0.1279
Epoch 5/50
 - 4s - loss: 0.1264
Epoch 6/50
 - 5s - loss: 0.1234
Epoch 7/50
 - 5s - loss: 0.1201
Epoch 8/50
 - 5s - loss: 0.1149
Epoch 9/50
 - 5s - loss: 0.1131
Epoch 10/50
 - 5s - loss: 0.1093
Epoch 11/50
 - 4s - loss: 0.1076
Epoch 12/50
 - 5s - loss: 0.1046
Epoch 13/50
 - 4s - loss: 0.1016
Epoch 14/50
 - 5s - loss: 0.1000
Epoch 15/50
 - 5s - loss: 0.0991
Epoch 16/50
 - 5s - loss: 0.0967
Epoch 17/50
 - 5s - loss: 0.0936
Epoch 18/50
 - 5s - loss: 0.0928
Epoch 19/50
 - 4s - loss: 0.0907
Epoch 20/50
 - 4s - loss: 0.0910
Epoch 21/50
 - 4s - loss: 0.0887
Epoch 22/50
 - 5s - loss: 0.0876
Epoch 23/50
 - 4s - loss: 0.0860
Epoch 24/50
 - 5s - loss: 0.0837
Epoch 25/50
 - 5s - loss: 0.0857
Epoch 26/50
 - 5s - loss: 0.0829
Epoch 27/50
 - 5s - loss: 0.0826
Epoch 28/50
 - 5s - loss: 0.0822
Epoch 29/50
 - 4s - loss: 0.0808
Epoc